In [1]:
import random
from operator import attrgetter
from collections import Counter

try:
    del Counter.__reduce___
except:
    pass

import numpy
from deap import algorithms, base, creator, tools

In [2]:
ITEMS = {'French Fries':(2.75,5),
        'Hot Wings':(3.55,7),
        'Mixed Fruit': (2.15,2),
        'Mozzarella Sticks': (4.2,4),
        'Sampler Plate': (5.8,10),
        'Side Salad': (3.35,3)}
ITEMS_NAME = list(ITEMS.keys())

In [3]:
creator.create("FitnessMulti", base.Fitness, weights = (1.0, -1.0, 1.0))

creator.create("Individual", Counter, fitness = creator.FitnessMulti)

IND_INIT_SIZE = 3

toolbox = base.Toolbox()
toolbox.register("attr_item",random.choice, ITEMS_NAME)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_item, IND_INIT_SIZE)
toolbox.register("population",tools.initRepeat,list,toolbox.individual)

In [4]:
toolbox.population(n=5)

[Individual({'Side Salad': 1, 'Sampler Plate': 1, 'Mozzarella Sticks': 1}),
 Individual({'Hot Wings': 1, 'Side Salad': 1, 'Mozzarella Sticks': 1}),
 Individual({'Mozzarella Sticks': 1, 'Sampler Plate': 1, 'Mixed Fruit': 1}),
 Individual({'Mozzarella Sticks': 1, 'Hot Wings': 1, 'Side Salad': 1}),
 Individual({'French Fries': 1, 'Hot Wings': 1, 'Side Salad': 1})]

In [5]:
def evalXKCD(individual, target_price):
    price = 0.0
    times = [0]
    food = 0
    for item, number in individual.items():
        if number > 0:
            price += ITEMS[item][0] * number
            times.append(ITEMS[item][1])
            food += number
    return (price - target_price),max(times),food

In [6]:
evalXKCD(creator.Individual({'French Fries':2,'Side Salad':2,'Sampler Plate':0}),15.05)

(-2.8500000000000014, 5, 4)

In [7]:
def cxCounter(ind1,ind2, indpb):
    for key in ITEMS.keys():
        if random.random() < indpb:
            ind1[key],ind2[key] =ind2[key],ind1[key]
        return ind1, ind2

In [8]:
cxCounter(creator.Individual({'French Fries':5,'Side Salad':2,'Sampler Plate':4}),
          creator.Individual({'French Fries':1,'Side Salad':3,'Sampler Plate':0}),
          0.5)

(Individual({'French Fries': 1, 'Side Salad': 2, 'Sampler Plate': 4}),
 Individual({'French Fries': 5, 'Side Salad': 3, 'Sampler Plate': 0}))

In [9]:
def mutCounter(individual):
    if random.random() > 0.5:
        individual.update([random.choice(ITEMS_NAME)])
    else:
        val = random.choice(ITEMS_NAME)
        individual.subtract([val])
        if individual[val]<0:
            del individual[val]
    return individual,

In [10]:
mutCounter(creator.Individual({'French Fries':5,'Side Salad':2,'Sampler Plate':4})
          )

(Individual({'French Fries': 5, 'Side Salad': 2, 'Sampler Plate': 4}),)

In [11]:
import sys

toolbox.register("evaluate",evalXKCD, target_price=15.05)
toolbox.decorate("evaluate",tools.DeltaPenalty(lambda ind: evalXKCD(ind,15.05)[0] <= 0,
                                              (-sys.float_info.max,
                                              sys.float_info.max,
                                              -sys.float_info.max)))
toolbox.register("mate",cxCounter,indpb=0.5)
toolbox.register("mutate",mutCounter)
toolbox.register("select",tools.selNSGA2)

In [12]:
NGEN = 100
MU = 50
LAMBDA =20
CXPB = 0.3
MUTPB = 0.3

pop = toolbox.population(n=MU)

In [13]:
hof =tools.ParetoFront()

price_stats = tools.Statistics(key = lambda ind: ind.fitness.values[0])
time_stats = tools.Statistics(key = lambda ind: ind.fitness.values[1])
food_stats = tools.Statistics(key = lambda ind: ind.fitness.values[2])

stats =tools.MultiStatistics(price=price_stats,time=time_stats,food=food_stats)
stats.register("avg",numpy.mean,axis=0)

algorithms.eaMuPlusLambda(pop,toolbox,MU,LAMBDA,CXPB,MUTPB,NGEN,stats,halloffame=hof)

   	      	             food             	            price             	             time             
   	      	------------------------------	------------------------------	------------------------------
gen	nevals	avg          	gen	nevals	avg          	gen	nevals	avg         	gen	nevals
0  	50    	-3.59539e+306	0  	50    	-3.59539e+306	0  	50    	3.59539e+306	0  	50    
1  	11    	3.02         	1  	11    	-4.56        	1  	11    	6.46        	1  	11    
2  	12    	2.98         	2  	12    	-4.462       	2  	12    	6.3         	2  	12    
3  	13    	2.96         	3  	13    	-4.42        	3  	13    	5.86        	3  	13    
4  	12    	3.02         	4  	12    	-3.973       	4  	12    	5.96        	4  	12    
5  	15    	3.08         	5  	15    	-3.864       	5  	15    	5.72        	5  	15    
6  	11    	3.2          	6  	11    	-4.388       	6  	11    	5           	6  	11    
7  	9     	3.3          	7  	9     	-3.321       	7  	9     	5.7         	7  	9     
8  	11    	3.44         	8 

94 	14    	3            	94 	14    	-0.1         	94 	14    	10          	94 	14    
95 	12    	3            	95 	12    	-0.1         	95 	12    	10          	95 	12    
96 	13    	3            	96 	13    	-0.1         	96 	13    	10          	96 	13    
97 	12    	3            	97 	12    	-0.1         	97 	12    	10          	97 	12    
98 	11    	3            	98 	11    	-0.1         	98 	11    	10          	98 	11    
99 	10    	3            	99 	10    	-0.1         	99 	10    	10          	99 	10    
100	7     	3            	100	7     	-0.1         	100	7     	10          	100	7     


([Individual({'Sampler Plate': 2, 'Side Salad': 1, 'French Fries': 0}),
  Individual({'Sampler Plate': 2, 'Side Salad': 1, 'French Fries': 0}),
  Individual({'Sampler Plate': 2, 'Side Salad': 1, 'French Fries': 0}),
  Individual({'Sampler Plate': 2, 'Side Salad': 1, 'French Fries': 0}),
  Individual({'Sampler Plate': 2, 'Side Salad': 1, 'French Fries': 0}),
  Individual({'Sampler Plate': 2, 'Side Salad': 1, 'French Fries': 0}),
  Individual({'Sampler Plate': 2, 'Side Salad': 1, 'French Fries': 0}),
  Individual({'Sampler Plate': 2, 'Side Salad': 1, 'French Fries': 0}),
  Individual({'Sampler Plate': 2, 'Side Salad': 1, 'French Fries': 0}),
  Individual({'Sampler Plate': 2, 'Side Salad': 1, 'French Fries': 0}),
  Individual({'Sampler Plate': 2, 'Side Salad': 1, 'French Fries': 0}),
  Individual({'Sampler Plate': 2, 'Side Salad': 1, 'French Fries': 0}),
  Individual({'Sampler Plate': 2, 'Side Salad': 1, 'French Fries': 0}),
  Individual({'Sampler Plate': 2, 'Side Salad': 1, 'French Fries

In [16]:
for i in hof:
    print(i, i.fitness)

Individual({'Sampler Plate': 2, 'Side Salad': 1, 'French Fries': 0}) ()
Individual({'Sampler Plate': 2, 'Side Salad': 1, 'Mozzarella Sticks': 1, 'French Fries': 0}) ()
Individual({'Sampler Plate': 2, 'Side Salad': 1, 'French Fries': 0}) ()
Individual({'Sampler Plate': 2, 'Side Salad': 1, 'French Fries': 0}) ()
Individual({'Sampler Plate': 2, 'Side Salad': 1, 'French Fries': 0}) ()
Individual({'Sampler Plate': 2, 'Side Salad': 1, 'French Fries': 0}) ()
Individual({'Sampler Plate': 2, 'Side Salad': 1, 'French Fries': 0}) ()
Individual({'Sampler Plate': 2, 'Side Salad': 1, 'French Fries': 0}) ()
Individual({'Sampler Plate': 2, 'Side Salad': 1, 'French Fries': 0}) ()
Individual({'Sampler Plate': 2, 'Side Salad': 1, 'French Fries': 0}) ()
Individual({'Sampler Plate': 2, 'Side Salad': 1, 'French Fries': 0}) ()
Individual({'Sampler Plate': 2, 'Side Salad': 1, 'French Fries': 0}) ()
Individual({'Sampler Plate': 2, 'Side Salad': 1, 'French Fries': 0}) ()
Individual({'Sampler Plate': 2, 'Side Sa

Individual({'Sampler Plate': 2, 'Side Salad': 1, 'French Fries': 0}) ()
Individual({'Sampler Plate': 2, 'Side Salad': 1}) ()
Individual({'Sampler Plate': 2, 'Side Salad': 1, 'French Fries': 0}) ()
Individual({'Sampler Plate': 2, 'Side Salad': 1}) ()
Individual({'Sampler Plate': 2, 'Side Salad': 1, 'French Fries': 1}) ()
Individual({'Sampler Plate': 2, 'Side Salad': 1, 'French Fries': 0}) ()
Individual({'Sampler Plate': 2, 'Side Salad': 2, 'French Fries': 0}) ()
Individual({'Sampler Plate': 2, 'Side Salad': 1}) ()
Individual({'Sampler Plate': 2, 'Side Salad': 1}) ()
Individual({'Sampler Plate': 2, 'Side Salad': 1, 'French Fries': 0}) ()
Individual({'Sampler Plate': 2, 'Side Salad': 1}) ()
Individual({'Sampler Plate': 2, 'Side Salad': 2, 'French Fries': 0}) ()
Individual({'Sampler Plate': 2, 'Side Salad': 1, 'French Fries': 0}) ()
Individual({'Sampler Plate': 2, 'Side Salad': 1, 'Mozzarella Sticks': 1, 'French Fries': 0}) ()
Individual({'Sampler Plate': 2, 'Side Salad': 1, 'French Fries'

Individual({'Sampler Plate': 2, 'Side Salad': 1, 'French Fries': 0}) ()
Individual({'Side Salad': 3, 'Mixed Fruit': 2, 'French Fries': 0}) ()
Individual({'Sampler Plate': 2, 'Side Salad': 1, 'French Fries': 0}) ()
Individual({'Sampler Plate': 2, 'Side Salad': 1, 'French Fries': 0}) ()
Individual({'Sampler Plate': 2, 'Side Salad': 1}) ()
Individual({'Sampler Plate': 2, 'Side Salad': 1}) ()
Individual({'Sampler Plate': 2, 'Side Salad': 1, 'French Fries': 0}) ()
Individual({'Sampler Plate': 2, 'Side Salad': 1, 'French Fries': 0}) ()
Individual({'Mixed Fruit': 3, 'Side Salad': 1, 'French Fries': 0}) ()
Individual({'Sampler Plate': 2, 'Side Salad': 1}) ()
Individual({'Sampler Plate': 2, 'Side Salad': 1, 'French Fries': 0}) ()
Individual({'Sampler Plate': 2, 'Side Salad': 1, 'French Fries': 0}) ()
Individual({'Sampler Plate': 2, 'Side Salad': 1, 'Mozzarella Sticks': 1}) ()
Individual({'Sampler Plate': 2, 'Side Salad': 1, 'French Fries': 0}) ()
Individual({'Sampler Plate': 2, 'Side Salad': 1}

Individual({'Mozzarella Sticks': 2, 'Sampler Plate': 1}) ()
Individual({'Side Salad': 2, 'Mozzarella Sticks': 1, 'Mixed Fruit': 1}) ()
Individual({'Side Salad': 2, 'Mixed Fruit': 1}) ()
Individual({'Sampler Plate': 2, 'Side Salad': 1, 'French Fries': 0}) ()
Individual({'Mozzarella Sticks': 2, 'Sampler Plate': 1, 'French Fries': 0}) ()
Individual({'Mozzarella Sticks': 1, 'Hot Wings': 1, 'Sampler Plate': 1, 'French Fries': 0}) ()
Individual({'Side Salad': 1, 'Mixed Fruit': 0}) ()
Individual({'Side Salad': 2, 'Mixed Fruit': 1}) ()
Individual({'Mixed Fruit': 1, 'Mozzarella Sticks': 1, 'French Fries': 1}) ()
Individual({'French Fries': 2, 'Mozzarella Sticks': 1}) ()
Individual({'Sampler Plate': 1, 'French Fries': 1, 'Mozzarella Sticks': 1}) ()
Individual({'French Fries': 2, 'Mozzarella Sticks': 1}) ()
Individual({'Sampler Plate': 2, 'Side Salad': 1}) ()
Individual({'Mixed Fruit': 1, 'Mozzarella Sticks': 1, 'French Fries': 0}) ()
Individual({'Hot Wings': 2, 'Mozzarella Sticks': 1, 'Mixed Fru

In [15]:
print("Best:",hof[0],hof[0].fitness.values)

Best: Individual({'Sampler Plate': 2, 'Side Salad': 1, 'French Fries': 0}) ()
